In [4]:
import pandas as pd
import glob 
import plotly.express as px 
import plotly.io as pio


In [5]:

# find all csv files in folder "Stock Data"
csv_files = glob.glob("/Users/jan/Techlabs_Project_Data/Data/*.csv") 
# read all csv files -> all files have the same structure
df_list = [pd.read_csv(file) for file in csv_files] 
#combine all csv files to one dataframe
stock_data = pd.concat(df_list, ignore_index=True)


In [6]:
#For POC we only use Data from Year 2008 & 2009 
stock_data['Date']=pd.to_datetime(stock_data['Date'], format= 'ISO8601', utc=True)
stock_data = stock_data[stock_data["Date"].dt.year<=2009]
#If we want to show the influence from Natural Disaster on stocks, we need the difference (in%) from Close to Open 
stock_data['percentage_difference_open_close'] = ((stock_data['Close']-stock_data['Open'])/stock_data['Open'])*100

In [7]:
#read data natural disaster
earthquakes_data = pd.read_csv('/Users/jan/Techlabs_Project_Data/Eartquakes-1990-2023.csv')


In [8]:
#For POC we only use Data from Year 2008 & 2009. 
earthquakes_data['date']=pd.to_datetime(earthquakes_data['date'], format= 'ISO8601')
earthquakes_data = earthquakes_data[(earthquakes_data['date'].dt.year>=2008)&(earthquakes_data['date'].dt.year<=2009)]
earthquakes_data=earthquakes_data.drop(columns=['time','status','tsunami'])# Drop columns 'time', 'status', 'tsunami'


In [9]:
# prepare date columns to act as primary key 
stock_data.rename(columns={'Date':'date'},inplace=True) #rename date columns in stock_data
# Make sure that both columns are in the same datetime format
earthquakes_data["date"] = pd.to_datetime(earthquakes_data["date"]).dt.date 
stock_data["date"] = pd.to_datetime(stock_data["date"]).dt.date

In [10]:
#merge stock_data and earthquake_data to one dataframe.primary key is the date
disaster_expanded = earthquakes_data.merge(stock_data, on = 'date', how= 'outer')
disaster_expanded = disaster_expanded.sort_values(by=["date"])


In [11]:
# Convert date to datetime
disaster_expanded["date"] = pd.to_datetime(disaster_expanded["date"])  



In [12]:
#Filter the Data to all Disasters in "State" Japan & Region Japan and Japan Ticker N225 or 
#if there was a natural disaster on a day when there was no trading day

disaster_japan = disaster_expanded[
    disaster_expanded["state"].str.contains('Japan', na=False) & 
    (disaster_expanded["Ticker"].eq('^N225') | disaster_expanded["Ticker"].isna())& (disaster_expanded['magnitudo']>= 6.0)]


In [13]:
disaster_japan.head()

,place,significance,data_type,magnitudo,state,longitude,latitude,depth,date,Ticker,Open,High,Low,Close,Adj Close,Volume,percentage_difference_open_close
17069,"Bonin Islands, Japan region",591.0,earthquake,6.2,Japan region,142.438,26.816,15.0,2008-02-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22995,"Bonin Islands, Japan region",554.0,earthquake,6.0,Japan region,142.599,26.988,10.0,2008-03-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53484,"78 km NE of Hasaki, Japan",910.0,earthquake,6.9,Japan,141.526,36.164,27.0,2008-05-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53451,"80 km NE of Hasaki, Japan",608.0,earthquake,6.2,Japan,141.545,36.178,19.0,2008-05-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53463,"95 km ENE of Hasaki, Japan",573.0,earthquake,6.1,Japan,141.756,36.156,23.3,2008-05-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
#define variabels that we want to use for the Info Points on the map 
latitude = disaster_japan["latitude"]
longitude = disaster_japan["longitude"]
magnitude = disaster_japan["magnitudo"]
depth = disaster_japan["depth"]
place = disaster_japan ['place']
date = disaster_japan ['date']

In [15]:
import yfinance as yf

In [16]:
#read data N225 Companies
nikkei_companies = pd.read_csv('/Users/jan/Techlabs_Project_Data/N225_companies.csv', sep=';', skiprows=1)

nikkei_companies.head()


,Number,Tickers,Company_name,Industry,Date,Open,Close,Volume
0,1,4151.T,"KYOWA KIRIN CO., LTD.",Pharmaceuticals,NaN,NaN,NaN,NaN
1,2,4502.T,"TAKEDA PHARMACEUTICAL CO., LTD.",Pharmaceuticals,NaN,NaN,NaN,NaN
2,3,4503.T,ASTELLAS PHARMA INC.,Pharmaceuticals,NaN,NaN,NaN,NaN
3,4,4506.T,"SUMITOMO PHARMA CO., LTD.",Pharmaceuticals,NaN,NaN,NaN,NaN
4,5,4507.T,"SHIONOGI & CO., LTD.",Pharmaceuticals,NaN,NaN,NaN,NaN


In [17]:
# Function to return adress from yahoo finance based on the ticker number 
def get_address_and_city(ticker):
    try:
        # Abrufen des Unternehmens über yfinance
        company = yf.Ticker(ticker)
        # Adresse und Stadt extrahieren
        address = company.info.get('address1', 'Adresse nicht verfügbar')
        city = company.info.get('city', 'Stadt nicht verfügbar')
        return address, city
    except Exception as e:
        print(f"Fehler beim Abrufen der Daten für {ticker}: {e}")
        return 'Fehler beim Abrufen', 'Fehler beim Abrufen'


# add new column to dataframe 
nikkei_companies[['Address', 'City']] = nikkei_companies['Tickers'].apply(lambda ticker: pd.Series(get_address_and_city(ticker)))

# print first lines to check if new column is correct
print(nikkei_companies[['Tickers', 'Company_name', 'Address', 'City']].head())

  Tickers                     Company_name  \
0  4151.T            KYOWA KIRIN CO., LTD.   
1  4502.T  TAKEDA PHARMACEUTICAL CO., LTD.   
2  4503.T             ASTELLAS PHARMA INC.   
3  4506.T        SUMITOMO PHARMA CO., LTD.   
4  4507.T             SHIONOGI & CO., LTD.   

                              Address   City  
0  Otemachi Financial City Grand Cube  Tokyo  
1      1-1, Nihonbashi-Honcho 2-chome  Tokyo  
2            2-5-1, Nihonbashi-Honcho  Tokyo  
3             6-8, Doshomachi 2-chome  Osaka  
4             1-8, Doshomachi 3-chome  Osaka  


In [18]:
# Open the map in the browser
pio.renderers.default = "browser"  
#Our Mapbox Access Token 
px.set_mapbox_access_token("pk.eyJ1IjoidGVjaGxhYnMzIiwiYSI6ImNtNzBib2xyczAwZHoycnBiM2hxZ24zcngifQ.AaSHNEfc-cnR4uDdEO4gsw")

In [19]:
import geocoder

def get_lat_lon(City):
    try:
        g = geocoder.mapbox(City, key="pk.eyJ1IjoidGVjaGxhYnMzIiwiYSI6ImNtNzBib2xyczAwZHoycnBiM2hxZ24zcngifQ.AaSHNEfc-cnR4uDdEO4gsw")  # Mapbox Geocoding
        if g.ok:
            return g.latlng[0], g.latlng[1]
        return None, None
    except Exception as e:
        print(f"Fehler bei {address}: {e}")
        return None, None

# Adressen in Latitude & Longitude umwandeln
nikkei_companies["latitude"], nikkei_companies["longitude"] = zip(*[get_lat_lon(addr) for addr in nikkei_companies["City"]])

# Ausgabe der Ergebnisse
nikkei_companies['City'].unique()

array(['Tokyo', 'Osaka', 'Chuo', 'Kitakyushu', 'Yokohama', 'Kyoto',
       'Suwa', 'Kadoma', 'Sakai', 'Musashino', 'Kariya', 'Yamanashi',
       'Nagaokakyo', 'Toyota', 'Hino', 'Hiroshima', 'Hamamatsu', 'Iwata',
       'Shinagawa', 'Nishitokyo', 'Shizuoka', 'Chiba City', 'Fukuoka',
       'Noda', 'Chiba', 'Yamaguchi', 'Urayasu', 'Chiyoda', 'Setagaya',
       'Hiratsuka', 'Otsu', 'Nagoya', 'Kobe', 'Niwa', 'Isehara', 'Taito',
       'Aichi', 'Ichikawa', 'Minato'], dtype=object)

In [20]:
nikkei_companies.head()

,Number,Tickers,Company_name,Industry,Date,Open,Close,Volume,Address,City,latitude,longitude
0,1,4151.T,"KYOWA KIRIN CO., LTD.",Pharmaceuticals,NaN,NaN,NaN,NaN,Otemachi Financial City Grand Cube,Tokyo,35.688820,139.692526
1,2,4502.T,"TAKEDA PHARMACEUTICAL CO., LTD.",Pharmaceuticals,NaN,NaN,NaN,NaN,"1-1, Nihonbashi-Honcho 2-chome",Tokyo,35.688820,139.692526
2,3,4503.T,ASTELLAS PHARMA INC.,Pharmaceuticals,NaN,NaN,NaN,NaN,"2-5-1, Nihonbashi-Honcho",Tokyo,35.688820,139.692526
3,4,4506.T,"SUMITOMO PHARMA CO., LTD.",Pharmaceuticals,NaN,NaN,NaN,NaN,"6-8, Doshomachi 2-chome",Osaka,34.683595,135.500784
4,5,4507.T,"SHIONOGI & CO., LTD.",Pharmaceuticals,NaN,NaN,NaN,NaN,"1-8, Doshomachi 3-chome",Osaka,34.683595,135.500784


In [21]:
pk.eyJ1IjoidGVjaGxhYnMzIiwiYSI6ImNtNzBib2xyczAwZHoycnBiM2hxZ24zcngifQ.AaSHNEfc-cnR4uDdEO4gsw

NameError: name 'pk' is not defined

In [91]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objects as go
import plotly.express as px
from dash.dependencies import Input, Output
import webbrowser
from threading import Timer
import pandas as pd

# 📌 Dash App erstellen
app = dash.Dash(__name__)

# 📌 Layout mit hellem Design & Filtern über der Map
app.layout = html.Div(
    style={"backgroundColor": "white", "color": "black", "padding": "20px", "minHeight": "100vh"},
    children=[
        html.H1(children='Stock_Market Analysis',
                style={'textAlign': 'left', "color": "#4A772F"}),

        # 🔽 Filter als Dropdowns über der Karte
# 🔽 Filter als Dropdowns über der Karte mit individuellen Überschriften
html.Div(
    style={"display": "flex", "justifyContent": "left", "gap": "20px", "marginBottom": "20px"},
    children=[
        html.Div([
            html.Label("Year", style={"fontSize": "15px", "color": "black"}),
            dcc.Dropdown(
                id='dropdown-selection',
                options=[{'label': date, 'value': date} for date in disaster_japan.date.unique()],
                value=disaster_japan.date.unique()[0],
                clearable=False,
                style={"backgroundColor": "white", "color": "black", "width": "250px"},
                placeholder="Wähle ein Datum",
            )
        ], style={"display": "flex", "flexDirection": "column"}),

        html.Div([
            html.Label("State", style={"fontSize": "15px", "color": "black"}),
            dcc.Dropdown(
                id='state-selection',
                options=[{'label': state, 'value': state} for state in disaster_japan.state.unique()],
                value=disaster_japan.state.unique()[0],
                clearable=False,
                style={"backgroundColor": "white", "color": "black", "width": "250px"},
                placeholder="Wähle einen State",
            )
        ], style={"display": "flex", "flexDirection": "column"}),

        html.Div([
            html.Label("Industry", style={"fontSize": "15px", "color": "black"}),
            dcc.Dropdown(
                id='industry-selection',
                options=[{'label': industry, 'value': industry} for industry in nikkei_companies.Industry.unique()],
                value=nikkei_companies.Industry.unique()[0],
                clearable=False,
                style={"backgroundColor": "white", "color": "black", "width": "250px"},
                placeholder="Wähle eine Industry",
            )
        ], style={"display": "flex", "flexDirection": "column"}),
    ],
),


        # 🌍 Map Bereich & Rechte Seite
        html.Div(
            id="main-container",
            style={"display": "flex", "gap": "20px", "height": "80vh"},  # Höhe setzen
            children=[
                html.Div(  # Map-Bereich (nimmt 50% ein)
                    id="map-container",
                    style={"flex": "0.5"},
                    children=[dcc.Graph(id='graph-content', style={"height": "100%"})],
                ),
                html.Div(  # Rechte Seite (nimmt 50% ein)
                    id="right-panel",
                    style={"flex": "0.5", "backgroundColor": "#f8f9fa", "padding": "20px", "borderRadius": "10px"},
                    children=[
                        html.H3("Weitere Informationen"),
                        html.P("Hier kannst du zusätzliche Inhalte einfügen."),
                    ],
                ),
            ],
        ),
    ]
)

# 🎯 Callback: Aktualisiert die Karte basierend auf den Filtern
@app.callback(
    Output('graph-content', 'figure'),
    [Input('dropdown-selection', 'value'),
     Input('state-selection', 'value'),
     Input('industry-selection', 'value')]
)
def update_map(selected_date, selected_state, selected_industry):
    # 🔍 Daten filtern
    filtered_data = disaster_japan[
        (disaster_japan['date'] == selected_date) &
        (disaster_japan['state'] == selected_state)
    ]
    # Unternehmens-Daten filtern
    filtered_companies = nikkei_companies[nikkei_companies['Industry'] == selected_industry]

    # 📊 Falls keine Daten vorhanden sind, leere Karte zurückgeben
    if filtered_data.empty and filtered_companies.empty:
        return go.Figure()

    # 🌍 Density Map mit CARTO-Style
    fig = px.density_mapbox(
        filtered_data,
        lat="latitude",
        lon="longitude",
        z="magnitudo",
        hover_name="place",
        radius=10,  # Dichte-Radius für bessere Sichtbarkeit
        zoom=5,
        mapbox_style="carto-positron",  # Setzt CARTO als Kartenstil
    )

    # Unternehmen als zusätzliche Marker hinzufügen
    if not filtered_companies.empty:
        fig.add_trace(go.Scattermapbox(
            lat=filtered_companies["latitude"].tolist(),
            lon=filtered_companies["longitude"].tolist(),
            mode="markers",
            marker=dict(size=15, color="cyan"),
            hovertext=filtered_companies["Company_name"],
            hoverinfo="text",
        
        ))

    # 🖌️ Styling & Design
    fig.update_layout(
        margin=dict(l=10, r=10, t=20, b=10, pad=5),
        plot_bgcolor="white",
        paper_bgcolor="white",
        clickmode="event+select",
        hovermode="closest",
        coloraxis_showscale=False,
        showlegend=False,
        mapbox=dict(
            accesstoken="pk.eyJ1IjoidGVjaGxhYnMzIiwiYSI6ImNtNzBib2xyczAwZHoycnBiM2hxZ24zcngifQ.AaSHNEfc-cnR4uDdEO4gsw",
            bearing=10,
            center=dict(
                lat=filtered_data.latitude.mean() if not filtered_data.empty else 50,
                lon=filtered_data.longitude.mean() if not filtered_data.empty else 10
            ),
            pitch=5,
            zoom=5,
        ),
    )

    return fig

# 🌍 Automatisches Öffnen des Browsers
def open_browser():
    webbrowser.open_new("http://127.0.0.1:8050/")

# 🚀 Starte die App im Browser
if __name__ == '__main__':
    Timer(1, open_browser).start()
    app.run_server(debug=True)


/var/folders/xm/s7wz86kd7szcj2pdp0nfk8mh0000gn/T/ipykernel_36505/2890700741.py:109: DeprecationWarning:

*density_mapbox* is deprecated! Use *density_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/

/var/folders/xm/s7wz86kd7szcj2pdp0nfk8mh0000gn/T/ipykernel_36505/2890700741.py:122: DeprecationWarning:

*scattermapbox* is deprecated! Use *scattermap* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/

